# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
total_schools = school_data_complete["school_name"].nunique()
total_students = school_data_complete["Student ID"].nunique()
total_budget = school_data["budget"].sum()
avg_math = school_data_complete["math_score"].mean()
avg_reading = school_data_complete["reading_score"].mean()
passing_rate = (avg_math + avg_reading)/2

In [3]:
passing_math = school_data_complete[school_data_complete["math_score"] >= 70]
passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70]

In [4]:
passing_math_stu = passing_math["student_name"].count()
passing_reading_stu = passing_reading["student_name"].count()

passing_math_percent = passing_math_stu / total_students * 100
passing_reading_percent = passing_reading_stu / total_students * 100

In [5]:
summary_dict = [{"Total Schools": total_schools,
                 "Total Students": total_students,
                 "Total Budget": total_budget,
                 "Average Math Score": avg_math,
                 "Average Reading Score": avg_reading,
                 "% Passing Math": passing_math_percent,
                 "% Passing Reading": passing_reading_percent,
                 "Overall Passing Rate": passing_rate}]

summary_df = pd.DataFrame(summary_dict)

summary_df = summary_df[["Total Schools", "Total Students", "Total Budget", "Average Math Score",
                         "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Passing Rate"]]

summary_df["Total Students"] = summary_df["Total Students"].astype(float).map("{:,.0f}".format)
summary_df["Total Budget"] = summary_df["Total Budget"].astype(float).map("${:,.0f}".format)
summary_df["Average Math Score"] = summary_df["Average Math Score"].astype(float).map("{:.2f}".format)
summary_df["Average Reading Score"] = summary_df["Average Reading Score"].astype(float).map("{:.2f}".format)
summary_df["% Passing Math"] = summary_df["% Passing Math"].astype(float).map("{:.2f}%".format)
summary_df["% Passing Reading"] = summary_df["% Passing Reading"].astype(float).map("{:.2f}%".format)
summary_df["Overall Passing Rate"] = summary_df["Overall Passing Rate"].astype(float).map("{:.2f}%".format)

summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428",78.99,81.88,74.98%,85.81%,80.43%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [6]:
school_group = school_data_complete.groupby(["school_name"])

In [7]:
total_math = school_group["math_score"].sum()
total_reading = school_group["reading_score"].sum()

In [8]:
passing_math_group = school_group["math_score"].apply(lambda x: x[x >= 70].count())
passing_reading_group = school_group["reading_score"].apply(lambda x: x[x >= 70].count())

In [9]:
school_group_info = pd.DataFrame({"Total Math": total_math,
                                  "Total Reading": total_reading,
                                  "Passing Math": passing_math_group,
                                  "Passing Reading": passing_reading_group})

In [10]:
school_data_merged = pd.merge(school_data, school_group_info, on="school_name")

In [11]:
school_data_merged["Per Student Budget"] = school_data_merged["budget"]/school_data_merged["size"]
school_data_merged["Average Math Score"] = school_data_merged["Total Math"]/school_data_merged["size"]
school_data_merged["Average Reading Score"] = school_data_merged["Total Reading"]/school_data_merged["size"]
school_data_merged["% Passing Math"] = school_data_merged["Passing Math"]/school_data_merged["size"] * 100
school_data_merged["% Passing Reading"] = school_data_merged["Passing Reading"]/school_data_merged["size"] * 100
school_data_merged["Overall Passing Rate"] = (school_data_merged["% Passing Math"] + school_data_merged["% Passing Reading"])/2

In [12]:
school_summary = school_data_merged.loc[:, ["school_name", "type", "size", "budget", "Per Student Budget",
                                            "Average Math Score", "Average Reading Score", "% Passing Math", 
                                            "% Passing Reading", "Overall Passing Rate"]]

school_summary = school_summary.rename(columns={"school_name": "School Name",
                                                "type": "School Type",
                                                "size": "Total Students",
                                                "budget": "Total School Budget"})

school_summary = school_summary.set_index("School Name")

school_summary_df = school_summary.copy()

school_summary_df["Total Students"] = school_summary_df["Total Students"].astype(float).map("{:,.0f}".format)
school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].astype(float).map("${:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].astype(float).map("${:,.2f}".format)
school_summary_df["Average Math Score"] = school_summary_df["Average Math Score"].astype(float).map("{:.2f}".format)
school_summary_df["Average Reading Score"] = school_summary_df["Average Reading Score"].astype(float).map("{:.2f}".format)
school_summary_df["% Passing Math"] = school_summary_df["% Passing Math"].astype(float).map("{:.2f}%".format)
school_summary_df["% Passing Reading"] = school_summary_df["% Passing Reading"].astype(float).map("{:.2f}%".format)
school_summary_df["Overall Passing Rate"] = school_summary_df["Overall Passing Rate"].astype(float).map("{:.2f}%".format)

school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,73.50%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,73.36%
Shelton High School,Charter,"1,761","$1,056,600.00",$600.00,83.36,83.73,93.87%,95.85%,94.86%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,73.81%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,95.27%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,95.20%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,95.59%
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,74.31%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,94.38%


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [13]:
top_schools = school_summary.sort_values("Overall Passing Rate", ascending=False)
top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [14]:
bottom_schools = school_summary.sort_values("Overall Passing Rate", ascending=True)
bottom_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [15]:
grade_9 = school_data_complete[school_data_complete["grade"] == "9th"]
grade_9_group = grade_9.groupby(["school_name"])

grade_10 = school_data_complete[school_data_complete["grade"] == "10th"]
grade_10_group = grade_10.groupby(["school_name"])

grade_11 = school_data_complete[school_data_complete["grade"] == "11th"]
grade_11_group = grade_11.groupby(["school_name"])

grade_12 = school_data_complete[school_data_complete["grade"] == "12th"]
grade_12_group = grade_12.groupby(["school_name"])

year9_avg = grade_9_group.mean()
year10_avg = grade_10_group.mean()
year11_avg = grade_11_group.mean()
year12_avg = grade_12_group.mean()

In [16]:
grades_merged = pd.merge(pd.merge(pd.merge(year9_avg, year10_avg, on="school_name", suffixes = ("_9","_10")), year11_avg, on="school_name"), year12_avg, on="school_name")

del grades_merged["Student ID_9"]
del grades_merged["School ID_9"]
del grades_merged["size_9"]
del grades_merged["budget_9"]
del grades_merged["Student ID_10"]
del grades_merged["School ID_10"]
del grades_merged["size_10"]
del grades_merged["budget_10"]
del grades_merged["Student ID_x"]
del grades_merged["School ID_x"]
del grades_merged["size_x"]
del grades_merged["budget_x"]
del grades_merged["Student ID_y"]
del grades_merged["School ID_y"]
del grades_merged["size_y"]
del grades_merged["budget_y"]

In [17]:
math_df = grades_merged[["math_score_9", "math_score_10", "math_score_x", "math_score_y"]]

math_df = math_df.rename(columns={"math_score_9": "9th",
                                  "math_score_10": "10th",
                                  "math_score_x": "11th",
                                  "math_score_y": "12th",})

math_df["9th"] = math_df["9th"].astype(float).map("{:.2f}".format)
math_df["10th"] = math_df["10th"].astype(float).map("{:.2f}".format)
math_df["11th"] = math_df["11th"].astype(float).map("{:.2f}".format)
math_df["12th"] = math_df["12th"].astype(float).map("{:.2f}".format)

math_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [18]:
reading_df = grades_merged[["reading_score_9", "reading_score_10", "reading_score_x", "reading_score_y"]]

reading_df = reading_df.rename(columns={"reading_score_9": "9th",
                                  "reading_score_10": "10th",
                                  "reading_score_x": "11th",
                                  "reading_score_y": "12th",})

reading_df["9th"] = reading_df["9th"].astype(float).map("{:.2f}".format)
reading_df["10th"] = reading_df["10th"].astype(float).map("{:.2f}".format)
reading_df["11th"] = reading_df["11th"].astype(float).map("{:.2f}".format)
reading_df["12th"] = reading_df["12th"].astype(float).map("{:.2f}".format)

reading_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [19]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [20]:
summary_spending = school_summary.copy()

In [21]:
summary_spending["Spending Range (Per Student)"] = pd.cut(summary_spending["Per Student Budget"], spending_bins, labels=group_names)

del summary_spending["School Type"]
del summary_spending["Total Students"]
del summary_spending["Total School Budget"]
del summary_spending["Per Student Budget"]

In [22]:
spending_groups = summary_spending.groupby(["Spending Range (Per Student)"])
spending_groups.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Range (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [23]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [24]:
summary_size = school_summary.copy()

In [25]:
summary_size["School Size"] = pd.cut(summary_size["Total Students"], size_bins, labels=group_names)

del summary_size["School Type"]
del summary_size["Total Students"]
del summary_size["Total School Budget"]
del summary_size["Per Student Budget"]

In [26]:
size_groups = summary_size.groupby(["School Size"])
size_groups.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [27]:
school_types = school_summary.copy()

del school_types["Total Students"]
del school_types["Total School Budget"]
del school_types["Per Student Budget"]

type_groups = school_types.groupby(["School Type"])

type_groups.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
